# Hands-On Assignment 5

In this assignment, you will practice everything that you have learned so far in an end-to-end setting.
You will be provided with a dataset that is **unique to you**, and your task is to perform
all the steps from previous assignments to clean, explore, visualize, and analyze your dataset.

**Written Portion**: Additionally, you will create a report that describes your process and provides insights about your dataset.
Each section that should appear in your report is noted with an orange star (like normal HO tasks).  The report should be  4-6 pages (12 pt font, 1.5 line spacing), and turned in on Canvas as a PDF.

The coding aspect for this assignment will be turned in the same was as all other HO's,
by submitting this file to the autograder.


For this assignment, feel free to make additional functions instead of implementing everything in the provided function.

The objective of this assignment is for you to apply and solidify the skills you have learned in previous assignments.

# Prompt

You have graduated from this class, and are a huge success!
You landed a job doing data science at some fancy company.

You just got a new client with some really interesting problems you get to solve.
Unfortunately, because of a big mess-up on their side the data's metadata got corrupted
(and the person that used to maintain the data just took a vow of silence and moved to a bog).

The only column you are sure about is the `label` column,
which contains a numeric label for each row.
Aside from that, the client does not know anything about the names, content, or even data types for each column.

Your task is to explore, clean, and analyze this data.
You should have already received an email with the details on obtaining your unique data.
Place it in the same directory as this notebook (and your `local_grader.py` script) and name it `data.txt`.

*I know this prompt may sound unrealistic, but I have literally been in a situation exactly like this.
I was working at a database startup, and one of our clients gave us data with over 70 columns and more than a million records and told us:
"The person who used to manage the data is no longer working with us, but this was the data they used to make all their decisions.
We also lost all the metadata information, like column names."
...
Working in industry is not always glamorous.
-Eriq*

# Part 0: Explore Your Data

Before you start doing things to/with your data, it's always a good idea to load up your data and take a look.

In [50]:
import pandas
import numpy
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.base import clone
from scipy.stats import ttest_ind


# Modify this to point to your data.
unique_data = pandas.read_csv('lucastel_data.txt', sep = "\t")
unique_data

,label,col_00,col_01,col_02,col_03,col_04,col_05,col_06,col_07,col_08,col_09,col_10,col_11,col_12,col_13
0,1,1194 m^2,0.6358,FREMONT,Ice Hockey,1298 kg,0.6776,983,234,888,0.8826,-0.1571,Ice Hockey,541,534
1,4,535 m^2,0.7506,Long Beach,tennis,484 kg,1.0805,976,822,795,0.6312,0.833,ice hockey,546,896
2,7,1173 m^2,-0.1847,San Francisco,boxing,964 kg,0.7395,1080,855,1375,0.5282,1.2588,"track & field, baseball",0,917
3,2,1205 m^2,1.064,Stockton,basketball,868 kg,0.4337,1004,779,177,0.5751,1.1442,"basketball, golf",858,1064
4,0,876 m^2,1.2449,Fresno,track & field,1024 kg,0.8037,839,867,998,1.1473,0.8599,tennis,1002,-25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1518,4,449 m^2,1.217,Long Beach,running,294 kg,1.0823,1081,543,985,0.6636,0.6569,motor sports,772,652
1519,3,870 m^2,1.1841,Santa Ana,badminton,1216 kg,0.9855,1230,605,1070,0.862,0.9887,volleyball,334,719
1520,6,722 m^2,NaN,Bakersfield,football,1359 kg,0.9593,480,928,763,1.2207,0.2474,boxing,699,842
1521,1,1032 m^2,0.4214,Fremont,ice hockey,1425 kg,0.7576,969,302,1009,1.2735,-0.0445,ice hockey,1375,487


Don't forget to checkout the column information.

In [44]:
unique_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1523 entries, 0 to 1522
Data columns (total 15 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   1523 non-null   int64 
 1   col_00  1512 non-null   object
 2   col_01  1507 non-null   object
 3   col_02  1514 non-null   object
 4   col_03  1513 non-null   object
 5   col_04  1509 non-null   object
 6   col_05  1510 non-null   object
 7   col_06  1514 non-null   object
 8   col_07  1510 non-null   object
 9   col_08  1512 non-null   object
 10  col_09  1510 non-null   object
 11  col_10  1512 non-null   object
 12  col_11  1511 non-null   object
 13  col_12  1506 non-null   object
 14  col_13  1511 non-null   object
dtypes: int64(1), object(14)
memory usage: 178.6+ KB


And any numeric information.

In [45]:
unique_data.describe()

,label
count,1523.000000
mean,3.498359
std,2.292756
min,0.000000
25%,2.000000
50%,4.000000
75%,5.000000
max,7.000000


<h4 style="color: darkorange; font-size: x-large";>★ Written Task: Introduction</h4>

Briefly describe the dataset you’re given and define the goal of the project and how you approach it.
For example, you can present a basic introduction of your data (shape and proposed data types)
and your goal is to use these features to predict the label of the response variable.
Then you propose a few models that are suitable for this project which will be introduced in the modeling section.

# Part 1: Data Cleaning

As always, we should start with data cleaning.
Take what you learned from HO3 to clean up this messy data to a point where it is ready for machine learning algorithms.

Some things you may want to do:
 - Deal with missing/empty values.
 - Fix numeric columns so that they actually contain numbers.
 - Remove inconsistencies from columns.
 - Assign a data type to each column.

<h4 style="color: darkorange; font-size: x-large";>★ Task 1.A</h4>

Complete the following function that takes in a DataFrame and outputs a clean version of the DataFrame.
You can assume that the frame has all the same structure as your unique dataset.
You can return the same or a new data frame.

In [56]:
def clean_data(frame):
    df = frame.copy()

    # any weird marks we just turn into simple NaN
    df = df.replace(['?', 'None', 'none', 'N/A', 'n/a'], numpy.nan)

    # strip any unnecessary units from any numeric-ish columns
    df['col_00'] = df['col_00'].str.replace(' m^2', '', regex=False)
    df['col_04'] = df['col_04'].str.replace(' kg', '', regex=False)

    numeric_cols = [
        'col_00', 'col_01', 'col_04', 'col_05', 'col_06',
        'col_07', 'col_08', 'col_09', 'col_10', 'col_12', 'col_13'
    ]

    # any numeric/coercing weird leftovers to Nan
    for col in numeric_cols:
        df[col] = pandas.to_numeric(df[col], errors='coerce')
    # clean up string/categorical columns
    df['col_02'] = df['col_02'].str.strip().str.title()
    df['col_03'] = df['col_03'].str.strip().str.lower()
    df['col_11'] = df['col_11'].str.strip().str.lower()

    # fill messing text with placeholder
    for col in ['col_02', 'col_03', 'col_11']:
        df[col] = df[col].fillna('unknown')
    # impute numeric Nans with column median
    for col in numeric_cols:
        df[col] = df[col].fillna(df[col].median())
    # one-hot encode categorical columns
    df = pandas.get_dummies(
        df,
        columns=['col_02', 'col_03', 'col_11'],
        drop_first=True
    )
    
    return df

unique_data = clean_data(unique_data)
unique_data

AttributeError: Can only use .str accessor with string values!

Now we should also be able to view all the numeric columns.

In [47]:
unique_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1523 entries, 0 to 1522
Data columns (total 70 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   label                           1523 non-null   int64  
 1   col_00                          1523 non-null   float64
 2   col_01                          1523 non-null   float64
 3   col_04                          1523 non-null   float64
 4   col_05                          1523 non-null   float64
 5   col_06                          1523 non-null   float64
 6   col_07                          1523 non-null   float64
 7   col_08                          1523 non-null   float64
 8   col_09                          1523 non-null   float64
 9   col_10                          1523 non-null   float64
 10  col_12                          1523 non-null   float64
 11  col_13                          1523 non-null   float64
 12  col_02_Bakersfield              15

<h4 style="color: darkorange; font-size: x-large";>★ Written Task: Data Cleaning</h4>

Describe the steps you took for data cleaning.
Why did you do this?
Did you have to make some choices along the way? If so, describe them.

# Part 2: Data Visualization

Once you have cleaned up the data, it is time to explore it and find interesting things.
Part of this exploration, will be visualizing the data in a way that makes it easier for yourself and others to understand.
Use what you have learned in HO1 and HO2 to create some visualizations for your dataset.

<h4 style="color: darkorange; font-size: x-large";>★ Written Task: Data Visualization</h4>

Create at least two different visualizations that help describe what you see in your dataset.
Include these visualizations in your report along with descriptions of
how you created the visualization,
what data preparation you had to do for the visualization (aside from the data cleaning in the previous part),
and what the visualization tells us about the data.

# Part 3: Modeling

Now that you have a good grasp of your clean data,
it is time to do some machine learning!
(Technically all our previous steps were also machine learning,
but now we get to use classifiers!)

Use the skills you developed to select **three** classifiers and implement them on your data.
For example, you can narrow down your choices to three classifiers which may include:
- Logistic regression
- K-nearest neighbors
- Decision tree
- Or others

<h4 style="color: darkorange; font-size: x-large";>★ Task 3.A</h4>

Complete the following function that takes in no parameters,
and returns a list with **three** untrained classifiers you are going to explore in this assignment.
This method may set parameters/options for the classifiers, but should not do any training/fitting.

For example, if you wanted to use logistic regression,
then **one** of your list items may be:
```
sklearn.linear_model.LogisticRegression()
```

In [51]:
def create_classifiers():
    clf1 = make_pipeline(
        StandardScaler(),
        LogisticRegression(max_iter=2000)
    )

    # k-NN (can also benefit from scaling if you want, but not required)
    clf2 = KNeighborsClassifier(n_neighbors=5)

    # Decision tree (does not care about scaling)
    clf3 = DecisionTreeClassifier(random_state=0)

    return [clf1, clf2, clf3]
    
    return [clf1, clf2, clf3]

my_classifiers = create_classifiers()
my_classifiers

[Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('logisticregression', LogisticRegression(max_iter=2000))]),
 KNeighborsClassifier(),
 DecisionTreeClassifier(random_state=0)]

Now that we have some classifiers, we can see how they perform.

<h4 style="color: darkorange; font-size: x-large";>★ Task 3.B</h4>

Complete the following function that takes in an untrained classifier, a DataFrame, and a number of folds.
This function should run k-fold cross validation with the classifier and the data,
and return a list with the accuracy of each run of cross validation.
You can assume that the frame has the column `label` and the rest of the columns can be considered clean numeric features.

Note that you may have to break your frame into features and labels to do this.
Do not change the passed-in frame (make copies instead).

If you are getting any `ConvergenceWarning`s you may either ignore them,
or try and address them
(they will not affect your autograder score, but may be something to discuss in the written portion of this assignment).

In [52]:
def cross_fold_validation(classifier, frame, folds):
    X = frame.drop(columns=['label'])
    y = frame['label']

    skf = StratifiedKFold(
        n_splits=folds,
        shuffle=True,
        random_state=0
    )

    scores = []
    for train_idx, test_idx in skf.split(X, y):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        # fresh copy of the classifier each fold
        clf = clone(classifier)
        clf.fit(X_train, y_train)

        y_pred = clf.predict(X_test)
        scores.append(accuracy_score(y_test, y_pred))

    return scores

my_classifiers_scores = []
for classifier in my_classifiers:
    accuracy_scores = cross_fold_validation(classifier, unique_data, 5)
    my_classifiers_scores.append(accuracy_scores)
    print("Classifier: %s, Accuracy: %s." % (type(classifier).__name__, accuracy_scores))

ValueError: could not convert string to float: '1194 m^2'

<h4 style="color: darkorange; font-size: x-large";>★ Task 3.C</h4>

Complete the following function that takes in two equally-sized lists of numbers and a p-value.
This function should compute whether there is a statistical significance between
these two lists of numbers using a [Student's t-test](https://en.wikipedia.org/wiki/Student%27s_t-test)
at the given p-value.
Return `True` if there is a statistical significance, and `False` otherwise.
Hint: If you wish, you may use the `ttest_ind()` [method](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html) provided in the scipy package. 

In [53]:
def significance_test(a_values, b_values, p_value):
    t_stat, p = ttest_ind(a_values, b_values, equal_var=False)
    return p < p_value

for i in range(len(my_classifiers)):
    for j in range(i + 1, len(my_classifiers)):
        significant = significance_test(my_classifiers_scores[i], my_classifiers_scores[j], 0.10)
        print("%s vs %s: %s" % (type(my_classifiers[i]).__name__,
                                type(my_classifiers[j]).__name__, significant))

IndexError: list index out of range

<h4 style="color: darkorange; font-size: x-large";>★ Written Task: Modeling</h4>

Describe the classifiers you have chosen.
Be sure to include all details about any parameter settings used for the algorithms.

Compare the performance of your models using k-fold validation.
You may look at accuracy, F1 or other measures.

Then, briefly summarize your results.
Are your results statistically significant?
Is there a clear winner?
What do the standard deviations look like, and what do they tell us about the different models?
Include a table like Table 1.

<center>Table 1: Every table needs a caption.</center>

| Model | Mean Accuracy | Standard Deviation of Accuracy |
|-------|---------------|--------------------------------|
| Logistic Regression | 0.724 | 0.004
| K-Nearest Neighbor | 0.750 | 0.003
| Decision Tree | 0.655 | 0.011

# Part 4: Analysis

Now, take some time to go over your results for each classifier and try to make sense of them.
 - Why do some classifiers work better than others?
 - Would another evaluation metric work better than vanilla accuracy?
 - Is there still a problem in the data that should be fixed in data cleaning?
 - Does the statistical significance between the different classifiers make sense?
 - Are there parameters for the classifier that I can tweak to get better performance?

<h4 style="color: darkorange; font-size: x-large";>★ Written Task: Analysis</h4>

Discuss your observations, the relationship you found, and how you applied concepts from the class to this project.
For example, you may find that some feature has the most impact in predicting your response variable or removing a feature improves the model accuracy.
Or you may observe that your training accuracy is much higher than your test accuracy and you may want to explain what issues may arise.

# Part 5: Conclusion

<h4 style="color: darkorange; font-size: x-large";>★ Written Task: Conclusion</h4>

Briefly summarize the important results and conclusions presented in the project.
What are the important points illustrated by your work?
Are there any areas for further investigation or improvement?

<h4 style="color: darkorange; font-size: x-large";>★ Written Task: References</h4>

Include a standard bibliography with citations referring to techniques or published papers you used throughout your report (if you used any).

For example:
```
[1] Derpanopoulos, G. (n.d.). Bayesian Model Checking & Comparison.
https://georgederpa.github.io/teaching/modelChecking.html.
```

# Part XC: Extra Credit

So far you have used a synthetic dataset that was created just for you.
But, data science is always more interesting when you are dealing with actual data from the real world.
Therefore, you will have an opportunity for extra credit on this assignment using real-world data.

For extra credit, repeat the **written tasks** of Parts 0 through 4 with an additional dataset that you find yourself.
For the written portion of the extra credit for Part 0, include information about where you got the data and what the data represents.
You may choose any dataset that represents real data (i.e., is **not** synthetic or generated)
and is **not** [pre-packaged in scikit-learn](https://scikit-learn.org/stable/datasets.html).

Below are some of the many places you can start looking for datasets:
 - [Kaggle](https://www.kaggle.com/datasets) -- Kaggle is a website focused around machine learning competitions,
       where people compete to see who can get the best results on a dataset.
       It is very popular in the machine learning community and has thousands of datasets with descriptions.
       Make sure to read the dataset's description, as Kaggle also has synthetic datasets.
 - [data.gov](https://data.gov/) -- A portal for data from the US government.
        The US government has a lot of data, and much of it has to be available to the public by law.
        This portal contains some of the more organized data from several different government agencies.
        In general, the government has A LOT of interesting data.
        It may not always be clean (remember the CIA factbook), but it is interesting and available.
        All data here should be real-world, but make sure to read the description to verify.
 - [UCI's Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets.php) -- UC Irvine has their own data repository with a few hundred datasets on many different topics.
        Make sure to read the dataset's description, as UCI also has synthetic datasets.
 - [WHO's Global Health Observatory](https://apps.who.int/gho/data/node.home) -- The World Health Organization keeps track of many different health-related statistics for most of the countries in the world.
        All data here should be real-world, but make sure to read the description to verify.
 - [Google's Dataset Search](https://datasetsearch.research.google.com/) -- Google indexes many datasets that can be searched here.

You can even create a dataset from scratch if you find some data you like that is not already organized into a specific dataset.
The only real distinction between "data" and a "dataset" is that a dataset is organized and finite (has a fixed size).

Create a new section in your written report for this extra credit and include all the written tasks for the extra credit there.
Each written task/section that you complete for your new dataset is eligible for extra credit (so you can still receive some extra credit even if you do not complete all parts).
There is no need to submit any code for the extra credit.
If you created a new dataset, include the dataset or links to it with your submission.